## Import packages

In [1]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder

In [2]:
# view all columns
pd.set_option('display.max_columns', None)

#### Dataset: [Telcom Customer Churn - Kaggle](https://www.kaggle.com/blastchar/telco-customer-churn)

In [3]:
dataset_path = r'.\dataset.csv'
df = pd.read_csv(dataset_path)
df

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,NaN,Yes,No,NaN,No,No phone service,DSL,No,Yes,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,NaN,No,No,34.0,Yes,No,DSL,Yes,No,Yes,No,No,No,One year,No,Mailed check,56.95,1889.50,No
2,3668-QPYBK,Male,NaN,No,No,2.0,Yes,No,DSL,Yes,Yes,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,Male,NaN,No,No,45.0,No,No phone service,DSL,Yes,No,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,9237-HQITU,Female,NaN,No,No,2.0,Yes,No,Fiber optic,No,No,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7038,6840-RESVB,Male,0.0,Yes,Yes,24.0,Yes,Yes,DSL,Yes,No,Yes,Yes,Yes,Yes,One year,Yes,Mailed check,84.80,1990.50,No
7039,2234-XADUH,Female,0.0,Yes,Yes,72.0,Yes,Yes,Fiber optic,No,Yes,Yes,No,Yes,Yes,One year,Yes,Credit card (automatic),103.20,7362.90,No
7040,4801-JZAZL,Female,0.0,Yes,Yes,11.0,No,No phone service,DSL,Yes,No,No,No,No,No,Month-to-month,Yes,Electronic check,29.60,346.45,No
7041,8361-LTMKD,Male,1.0,Yes,No,4.0,Yes,Yes,Fiber optic,No,No,No,No,No,No,Month-to-month,Yes,Mailed check,74.40,306.60,Yes


## Assessing the Data.

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 21 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   customerID        7043 non-null   object 
 1   gender            7043 non-null   object 
 2   SeniorCitizen     7000 non-null   float64
 3   Partner           7043 non-null   object 
 4   Dependents        7043 non-null   object 
 5   tenure            6896 non-null   float64
 6   PhoneService      7043 non-null   object 
 7   MultipleLines     7043 non-null   object 
 8   InternetService   7043 non-null   object 
 9   OnlineSecurity    7043 non-null   object 
 10  OnlineBackup      7043 non-null   object 
 11  DeviceProtection  7043 non-null   object 
 12  TechSupport       7043 non-null   object 
 13  StreamingTV       7043 non-null   object 
 14  StreamingMovies   7043 non-null   object 
 15  Contract          7043 non-null   object 
 16  PaperlessBilling  7043 non-null   object 


### Checking columns with missing values 

In [5]:
# how many missing values
df.isnull().sum().sort_values(ascending= False)[:2]

tenure           147
SeniorCitizen     43
dtype: int64

### Checking duplicates


In [6]:
# duplicate checking.
df.duplicated().sum()

0

- Feature Selection 
- Missing Values
- Outliers
- Label Encoding (Categorical Data)
- Feature Scaling 

# Data Cleaning

In [7]:
cleansed_df = df.copy()

### Feature Selection

In [8]:
cleansed_df.drop(['customerID'], axis = 1, inplace = True)

### Missing Values & Outliers

In [9]:
cleansed_df.SeniorCitizen.value_counts()

0.0    5858
1.0    1142
Name: SeniorCitizen, dtype: int64

In [10]:
maximum_value = cleansed_df.SeniorCitizen.value_counts().idxmax()
maximum_value

0.0

In [11]:
cleansed_df.SeniorCitizen.fillna(maximum_value, inplace = True)
# SeniorCitizen Column values represents an integer value, we should convert it to an integer not float.
cleansed_df['SeniorCitizen'] = cleansed_df.SeniorCitizen.astype(int)

In [12]:
cleansed_df.isnull().sum()

gender                0
SeniorCitizen         0
Partner               0
Dependents            0
tenure              147
PhoneService          0
MultipleLines         0
InternetService       0
OnlineSecurity        0
OnlineBackup          0
DeviceProtection      0
TechSupport           0
StreamingTV           0
StreamingMovies       0
Contract              0
PaperlessBilling      0
PaymentMethod         0
MonthlyCharges        0
TotalCharges          0
Churn                 0
dtype: int64

Tenure columns can be re-assigned by dividing total charges over monthly charges, with this way,
we can elminate outliers and missing values in one shot.

In [13]:
cleansed_df['tenure'] = round((cleansed_df.TotalCharges/cleansed_df.MonthlyCharges), 0).astype(int)

In [14]:
cleansed_df.isnull().sum()

gender              0
SeniorCitizen       0
Partner             0
Dependents          0
tenure              0
PhoneService        0
MultipleLines       0
InternetService     0
OnlineSecurity      0
OnlineBackup        0
DeviceProtection    0
TechSupport         0
StreamingTV         0
StreamingMovies     0
Contract            0
PaperlessBilling    0
PaymentMethod       0
MonthlyCharges      0
TotalCharges        0
Churn               0
dtype: int64

### Label Encoding (Categorical Data)

In [15]:
def encode_columns(df):
    mapped_columns = {}
    columns_to_be_labeled = df.loc[:, df.dtypes == object].columns
    
    for column in columns_to_be_labeled:
        labeled_column = LabelEncoder()
        df[column] = labeled_column.fit_transform(df[column])
        mapped_columns[column] = labeled_column
        
    return mapped_columns

In [16]:
labeled_columns = encode_columns(cleansed_df)
labeled_columns

{'gender': LabelEncoder(),
 'Partner': LabelEncoder(),
 'Dependents': LabelEncoder(),
 'PhoneService': LabelEncoder(),
 'MultipleLines': LabelEncoder(),
 'InternetService': LabelEncoder(),
 'OnlineSecurity': LabelEncoder(),
 'OnlineBackup': LabelEncoder(),
 'DeviceProtection': LabelEncoder(),
 'TechSupport': LabelEncoder(),
 'StreamingTV': LabelEncoder(),
 'StreamingMovies': LabelEncoder(),
 'Contract': LabelEncoder(),
 'PaperlessBilling': LabelEncoder(),
 'PaymentMethod': LabelEncoder(),
 'Churn': LabelEncoder()}

In [17]:
labeled_columns['gender'].classes_

array(['Female', 'Male'], dtype=object)

In [18]:
labeled_columns['PaymentMethod'].classes_

array(['Bank transfer (automatic)', 'Credit card (automatic)',
       'Electronic check', 'Mailed check'], dtype=object)

In [19]:
cleansed_df.head()

,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,0,0,1,0,1,0,1,0,0,2,0,0,0,0,0,1,2,29.85,29.85,0
1,1,0,0,0,33,1,0,0,2,0,2,0,0,0,1,0,3,56.95,1889.50,0
2,1,0,0,0,2,1,0,0,2,2,0,0,0,0,0,1,3,53.85,108.15,1
3,1,0,0,0,44,0,1,0,2,0,2,2,0,0,1,0,0,42.30,1840.75,0
4,0,0,0,0,2,1,0,1,0,0,0,0,0,0,0,1,2,70.70,151.65,1


### Feature Scaling
Make all numeric values in the same range

In [20]:
cleansed_df['tenure'] = (cleansed_df.tenure - cleansed_df.tenure.min())/(cleansed_df.tenure.max() - cleansed_df.tenure.min())
cleansed_df['TotalCharges'] = (cleansed_df.TotalCharges - cleansed_df.TotalCharges.min())/(cleansed_df.TotalCharges.max() - cleansed_df.TotalCharges.min())
cleansed_df['MonthlyCharges'] = (cleansed_df.MonthlyCharges - cleansed_df.MonthlyCharges.min())/(cleansed_df.MonthlyCharges.max() - cleansed_df.MonthlyCharges.min())

In [21]:
cleansed_df

,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,0,0,1,0,0.000000,0,1,0,0,2,0,0,0,0,0,1,2,0.115423,0.001275,0
1,1,0,0,0,0.278261,1,0,0,2,0,2,0,0,0,1,0,3,0.385075,0.215867,0
2,1,0,0,0,0.008696,1,0,0,2,2,0,0,0,0,0,1,3,0.354229,0.010310,1
3,1,0,0,0,0.373913,0,1,0,2,0,2,2,0,0,1,0,0,0.239303,0.210241,0
4,0,0,0,0,0.008696,1,0,1,0,0,0,0,0,0,0,1,2,0.521891,0.015330,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7038,1,0,1,1,0.191304,1,2,0,2,0,2,2,2,2,1,1,3,0.662189,0.227521,0
7039,0,0,1,1,0.608696,1,2,1,0,2,2,0,2,2,1,1,1,0.845274,0.847461,0
7040,0,0,1,1,0.095652,0,1,0,2,0,0,0,0,0,0,1,2,0.112935,0.037809,0
7041,1,1,1,0,0.026087,1,2,1,0,0,0,0,0,0,0,1,3,0.558706,0.033210,1


### Saving the dataset to a new csv file.

In [22]:
cleansed_df.to_csv('cleansed_dataset.csv', index = False)